# Buchwald-Hartwig HTE Dataset

For the Buchwald-Hartwig reactions we used the data set and splits as provided by

Sandfort et al. "A structure-based platform for predicting chemical reactivity." Chem (2020).
and originally published in

Ahneman et al. "Predicting reaction performance in C–N cross-coupling using machine learning." Science 360.6385 (2018): 186-190.

In [3]:
from rdkit import Chem
from rdkit.Chem import rdChemReactions

In [4]:
import pandas as pd
df_doyle = pd.read_excel('Dreher_and_Doyle_input_data.xlsx')

In [5]:
df_doyle.head()

,Ligand,Additive,Base,Aryl halide,Output
0,CC(C)C(C=C(C(C)C)C=C1C(C)C)=C1C2=C(P([C@@]3(C[...,CC1=CC(C)=NO1,CN(C)P(N(C)C)(N(C)C)=NP(N(C)C)(N(C)C)=NCC,ClC1=NC=CC=C1,70.410458
1,CC(C)C(C=C(C(C)C)C=C1C(C)C)=C1C2=C(P([C@@]3(C[...,O=C(OC)C1=CC=NO1,CN(C)P(N(C)C)(N(C)C)=NP(N(C)C)(N(C)C)=NCC,BrC1=NC=CC=C1,11.064457
2,CC(C)C(C=C(C(C)C)C=C1C(C)C)=C1C2=C(P(C3CCCCC3)...,O=C(OC)C1=CC=NO1,CN(C)P(N(C)C)(N(C)C)=NP(N(C)C)(N(C)C)=NCC,IC1=CC=C(CC)C=C1,10.223550
3,CC(C)C(C=C(C(C)C)C=C1C(C)C)=C1C2=C(P(C(C)(C)C)...,CCOC(C1=CON=C1)=O,CN1CCCN2C1=NCCC2,ClC1=CC=C(C(F)(F)F)C=C1,20.083383
4,CC(C)C(C=C(C(C)C)C=C1C(C)C)=C1C2=C(P([C@@]3(C[...,CC1=CC(C)=NO1,CN1CCCN2C1=NCCC2,ClC1=CC=C(OC)C=C1,0.492663


# Get Reaction Smiles

In [6]:
def canonicalize_with_dict(smi, can_smi_dict={}):
    if smi not in can_smi_dict.keys():
        return Chem.MolToSmiles(Chem.MolFromSmiles(smi))
    else:
        return can_smi_dict[smi]

In [7]:
def generate_buchwald_hartwig_rxns(df):
    """
    Converts the entries in the excel files from Sandfort et al. to reaction SMILES.
    """
    df = df.copy()
    fwd_template = '[F,Cl,Br,I]-[c;H0;D3;+0:1](:[c,n:2]):[c,n:3].[NH2;D1;+0:4]-[c:5]>>[c,n:2]:[c;H0;D3;+0:1](:[c,n:3])-[NH;D2;+0:4]-[c:5]'
    methylaniline = 'Cc1ccc(N)cc1'
    pd_catalyst = Chem.MolToSmiles(Chem.MolFromSmiles('O=S(=O)(O[Pd]1~[NH2]C2C=CC=CC=2C2C=CC=CC1=2)C(F)(F)F'))
    methylaniline_mol = Chem.MolFromSmiles(methylaniline)
    rxn = rdChemReactions.ReactionFromSmarts(fwd_template)
    products = []
    for i, row in df.iterrows():
        reacts = (Chem.MolFromSmiles(row['Aryl halide']), methylaniline_mol)
        rxn_products = rxn.RunReactants(reacts)

        rxn_products_smiles = set([Chem.MolToSmiles(mol[0]) for mol in rxn_products])
        assert len(rxn_products_smiles) == 1
        products.append(list(rxn_products_smiles)[0])
    df['product'] = products
    rxns = []
    can_smiles_dict = {}
    for i, row in df.iterrows():
        aryl_halide = canonicalize_with_dict(row['Aryl halide'], can_smiles_dict)
        can_smiles_dict[row['Aryl halide']] = aryl_halide
        ligand = canonicalize_with_dict(row['Ligand'], can_smiles_dict)
        can_smiles_dict[row['Ligand']] = ligand
        base = canonicalize_with_dict(row['Base'], can_smiles_dict)
        can_smiles_dict[row['Base']] = base
        additive = canonicalize_with_dict(row['Additive'], can_smiles_dict)
        can_smiles_dict[row['Additive']] = additive

        reactants = f"{aryl_halide}.{methylaniline}.{pd_catalyst}.{ligand}.{base}.{additive}"
        rxns.append(f"{reactants}>>{row['product']}")
    return rxns

In [8]:
df_doyle['rxn'] = generate_buchwald_hartwig_rxns(df_doyle)

In [9]:
df_doyle

,Ligand,Additive,Base,Aryl halide,Output,rxn
0,CC(C)C(C=C(C(C)C)C=C1C(C)C)=C1C2=C(P([C@@]3(C[...,CC1=CC(C)=NO1,CN(C)P(N(C)C)(N(C)C)=NP(N(C)C)(N(C)C)=NCC,ClC1=NC=CC=C1,70.410458,Clc1ccccn1.Cc1ccc(N)cc1.O=S(=O)(O[Pd]1c2ccccc2...
1,CC(C)C(C=C(C(C)C)C=C1C(C)C)=C1C2=C(P([C@@]3(C[...,O=C(OC)C1=CC=NO1,CN(C)P(N(C)C)(N(C)C)=NP(N(C)C)(N(C)C)=NCC,BrC1=NC=CC=C1,11.064457,Brc1ccccn1.Cc1ccc(N)cc1.O=S(=O)(O[Pd]1c2ccccc2...
2,CC(C)C(C=C(C(C)C)C=C1C(C)C)=C1C2=C(P(C3CCCCC3)...,O=C(OC)C1=CC=NO1,CN(C)P(N(C)C)(N(C)C)=NP(N(C)C)(N(C)C)=NCC,IC1=CC=C(CC)C=C1,10.223550,CCc1ccc(I)cc1.Cc1ccc(N)cc1.O=S(=O)(O[Pd]1c2ccc...
3,CC(C)C(C=C(C(C)C)C=C1C(C)C)=C1C2=C(P(C(C)(C)C)...,CCOC(C1=CON=C1)=O,CN1CCCN2C1=NCCC2,ClC1=CC=C(C(F)(F)F)C=C1,20.083383,FC(F)(F)c1ccc(Cl)cc1.Cc1ccc(N)cc1.O=S(=O)(O[Pd...
4,CC(C)C(C=C(C(C)C)C=C1C(C)C)=C1C2=C(P([C@@]3(C[...,CC1=CC(C)=NO1,CN1CCCN2C1=NCCC2,ClC1=CC=C(OC)C=C1,0.492663,COc1ccc(Cl)cc1.Cc1ccc(N)cc1.O=S(=O)(O[Pd]1c2cc...
...,...,...,...,...,...,...
3950,CC(C)C(C=C(C(C)C)C=C1C(C)C)=C1C2=C(P(C3CCCCC3)...,C1(C2=CC=CC=C2)=CON=C1,CN(C)P(N(C)C)(N(C)C)=NP(N(C)C)(N(C)C)=NCC,BrC1=CC=C(OC)C=C1,4.344677,COc1ccc(Br)cc1.Cc1ccc(N)cc1.O=S(=O)(O[Pd]1c2cc...
3951,CC(C)C(C=C(C(C)C)C=C1C(C)C)=C1C2=C(P(C(C)(C)C)...,CC1=CC(N2C=CC=C2)=NO1,CN1CCCN2C1=NCCC2,BrC1=CC=C(OC)C=C1,47.156275,COc1ccc(Br)cc1.Cc1ccc(N)cc1.O=S(=O)(O[Pd]1c2cc...
3952,CC(C)C(C=C(C(C)C)C=C1C(C)C)=C1C2=C(P(C(C)(C)C)...,CCOC(C1=CON=C1)=O,CC(C)(C)/N=C(N(C)C)/N(C)C,ClC1=CC=C(C(F)(F)F)C=C1,0.701552,FC(F)(F)c1ccc(Cl)cc1.Cc1ccc(N)cc1.O=S(=O)(O[Pd...
3953,CC(C)C(C=C(C(C)C)C=C1C(C)C)=C1C2=C(P([C@@]3(C[...,C1(N(CC2=CC=CC=C2)CC3=CC=CC=C3)=CC=NO1,CN(C)P(N(C)C)(N(C)C)=NP(N(C)C)(N(C)C)=NCC,ClC1=CC=C(C(F)(F)F)C=C1,15.561565,FC(F)(F)c1ccc(Cl)cc1.Cc1ccc(N)cc1.O=S(=O)(O[Pd...


# Get Mapped Reaction Smiles

In [10]:
from indigo import *
indigo = Indigo()

In [11]:
def gen_rdkit2d_features(sm):
    generator = rdNormalizedDescriptors.RDKit2DNormalized()
    try:
        features = generator.process(sm)[1:]
    except TypeError as e:
        return [0.] * 200
    return features

In [12]:
count = 0
def complete_atom_mapping(r):
    
    global count
    if pd.isnull(r):
        return None
    str_r, str_p = r.split('>>')
    mr = Chem.MolFromSmiles(str_r)
    mp = Chem.MolFromSmiles(str_p)
    
    for m in [mr, mp]:
        unique_indices = [0]
        for a in m.GetAtoms():
            if a.GetAtomMapNum() in unique_indices:
                a.SetAtomMapNum(0)
            else:
                unique_indices.append(a.GetAtomMapNum())

    unmapped_symbols_r = [a  for a in mr.GetAtoms() if a.GetAtomMapNum() == 0]
    unmapped_symbols_p = [a  for a in mp.GetAtoms() if a.GetAtomMapNum() == 0]
    #maybe add some kind of substucture mapping
    str_to_add_r, str_to_add_p = '', ''
    max_indx = max(max([a.GetAtomMapNum() for a in mr.GetAtoms()]), max([a.GetAtomMapNum() for a in mp.GetAtoms()]))
    for a_r in unmapped_symbols_r:
        was_mapped = False
        s_r = a_r.GetSymbol()
        a_r.SetAtomMapNum(max_indx + 1)
        for i, a_p in enumerate(unmapped_symbols_p):
            if s_r == a_p.GetSymbol():
                a_p.SetAtomMapNum(max_indx + 1)
                unmapped_symbols_p.pop(i)
                was_mapped = True
                break
        
        if not was_mapped:
            str_to_add_p += f'.[{a_r.GetSymbol()}:{max_indx + 1}]'
        max_indx += 1
    
    for a_p in unmapped_symbols_p:
        a_p.SetAtomMapNum(max_indx + 1)
        str_to_add_r += f'.[{a_p.GetSymbol()}:{max_indx + 1}]'
        max_indx += 1
    
    str_r = Chem.MolToSmiles(mr) + str_to_add_r
    str_p = Chem.MolToSmiles(mp) + str_to_add_p
    
    if (not Chem.MolFromSmiles(str_r)) or (not Chem.MolFromSmiles(str_p)):
        return None
    
    if Chem.MolFromSmiles(str_r).GetNumAtoms() !=  Chem.MolFromSmiles(str_p).GetNumAtoms():
        count += 1
#         print('>>'.join([str_r, str_p]))
#         print(Chem.MolFromSmiles(str_r).GetNumAtoms(), Chem.MolFromSmiles(str_p).GetNumAtoms())
#         print()
#         return None

    return '>>'.join([str_r, str_p])

In [13]:
df_doyle['mapped_rxn'] = df_doyle['rxn'].apply(lambda rxn:complete_atom_mapping(rxn))

# Get Labeled Dataset


In [14]:
def get_changed_bonds(rxn_smi):
    reactants = Chem.MolFromSmiles(rxn_smi.split('>')[0])
    products  = Chem.MolFromSmiles(rxn_smi.split('>')[2])

    conserved_maps = [a.GetProp('molAtomMapNumber') for a in products.GetAtoms() if a.HasProp('molAtomMapNumber')]
    bond_changes = set() # keep track of bond changes

    # Look at changed bonds
    bonds_prev = {}
    for bond in reactants.GetBonds():
        nums = sorted(
            [bond.GetBeginAtom().GetProp('molAtomMapNumber'),
             bond.GetEndAtom().GetProp('molAtomMapNumber')])
        if (nums[0] not in conserved_maps) and (nums[1] not in conserved_maps): continue
        bonds_prev['{}~{}'.format(nums[0], nums[1])] = bond.GetBondTypeAsDouble()
    bonds_new = {}
    for bond in products.GetBonds():
        nums = sorted(
            [bond.GetBeginAtom().GetProp('molAtomMapNumber'),
             bond.GetEndAtom().GetProp('molAtomMapNumber')])
        bonds_new['{}~{}'.format(nums[0], nums[1])] = bond.GetBondTypeAsDouble()


    for bond in bonds_prev:
        if bond not in bonds_new:
            bond_changes.add((bond.split('~')[0], bond.split('~')[1], 0.0)) # lost bond
        else:
            if bonds_prev[bond] != bonds_new[bond]:
                bond_changes.add((bond.split('~')[0], bond.split('~')[1], bonds_new[bond])) # changed bond
    for bond in bonds_new:
        if bond not in bonds_prev:
            bond_changes.add((bond.split('~')[0], bond.split('~')[1], bonds_new[bond]))  # new bond

    return bond_changes



In [15]:
df_doyle['edit'] = df_doyle['mapped_rxn'].apply(lambda map_rxn:get_changed_bonds(map_rxn))

In [16]:
df_doyle.drop('Output',inplace=True, axis=1)

In [17]:
df_doyle

,Ligand,Additive,Base,Aryl halide,rxn,mapped_rxn,edit
0,CC(C)C(C=C(C(C)C)C=C1C(C)C)=C1C2=C(P([C@@]3(C[...,CC1=CC(C)=NO1,CN(C)P(N(C)C)(N(C)C)=NP(N(C)C)(N(C)C)=NCC,ClC1=NC=CC=C1,Clc1ccccn1.Cc1ccc(N)cc1.O=S(=O)(O[Pd]1c2ccccc2...,[CH3:105][c:106]1[cH:107][c:108]([CH3:109])[o:...,"{(53, 54, 0.0), (86, 87, 0.0), (88, 89, 0.0), ..."
1,CC(C)C(C=C(C(C)C)C=C1C(C)C)=C1C2=C(P([C@@]3(C[...,O=C(OC)C1=CC=NO1,CN(C)P(N(C)C)(N(C)C)=NP(N(C)C)(N(C)C)=NCC,BrC1=NC=CC=C1,Brc1ccccn1.Cc1ccc(N)cc1.O=S(=O)(O[Pd]1c2ccccc2...,[Br:1][c:2]1[cH:3][cH:4][cH:5][cH:6][n:7]1.[CH...,"{(53, 54, 0.0), (86, 87, 0.0), (88, 89, 0.0), ..."
2,CC(C)C(C=C(C(C)C)C=C1C(C)C)=C1C2=C(P(C3CCCCC3)...,O=C(OC)C1=CC=NO1,CN(C)P(N(C)C)(N(C)C)=NP(N(C)C)(N(C)C)=NCC,IC1=CC=C(CC)C=C1,CCc1ccc(I)cc1.Cc1ccc(N)cc1.O=S(=O)(O[Pd]1c2ccc...,[CH3:10][c:11]1[cH:12][cH:13][c:14]([NH2:15])[...,"{(53, 54, 0.0), (86, 87, 0.0), (23, 24, 0.0), ..."
3,CC(C)C(C=C(C(C)C)C=C1C(C)C)=C1C2=C(P(C(C)(C)C)...,CCOC(C1=CON=C1)=O,CN1CCCN2C1=NCCC2,ClC1=CC=C(C(F)(F)F)C=C1,FC(F)(F)c1ccc(Cl)cc1.Cc1ccc(N)cc1.O=S(=O)(O[Pd...,[CH3:12][c:13]1[cH:14][cH:15][c:16]([NH2:17])[...,"{(88, 89, 0.0), (23, 24, 0.0), (33, 34, 0.0), ..."
4,CC(C)C(C=C(C(C)C)C=C1C(C)C)=C1C2=C(P([C@@]3(C[...,CC1=CC(C)=NO1,CN1CCCN2C1=NCCC2,ClC1=CC=C(OC)C=C1,COc1ccc(Cl)cc1.Cc1ccc(N)cc1.O=S(=O)(O[Pd]1c2cc...,[CH3:10][c:11]1[cH:12][cH:13][c:14]([NH2:15])[...,"{(53, 54, 0.0), (86, 87, 0.0), (88, 89, 0.0), ..."
...,...,...,...,...,...,...,...
3950,CC(C)C(C=C(C(C)C)C=C1C(C)C)=C1C2=C(P(C3CCCCC3)...,C1(C2=CC=CC=C2)=CON=C1,CN(C)P(N(C)C)(N(C)C)=NP(N(C)C)(N(C)C)=NCC,BrC1=CC=C(OC)C=C1,COc1ccc(Br)cc1.Cc1ccc(N)cc1.O=S(=O)(O[Pd]1c2cc...,[CH3:10][c:11]1[cH:12][cH:13][c:14]([NH2:15])[...,"{(53, 54, 0.0), (86, 87, 0.0), (23, 24, 0.0), ..."
3951,CC(C)C(C=C(C(C)C)C=C1C(C)C)=C1C2=C(P(C(C)(C)C)...,CC1=CC(N2C=CC=C2)=NO1,CN1CCCN2C1=NCCC2,BrC1=CC=C(OC)C=C1,COc1ccc(Br)cc1.Cc1ccc(N)cc1.O=S(=O)(O[Pd]1c2cc...,[CH3:10][c:11]1[cH:12][cH:13][c:14]([NH2:15])[...,"{(86, 87, 0.0), (50, 52, 0.0), (88, 89, 0.0), ..."
3952,CC(C)C(C=C(C(C)C)C=C1C(C)C)=C1C2=C(P(C(C)(C)C)...,CCOC(C1=CON=C1)=O,CC(C)(C)/N=C(N(C)C)/N(C)C,ClC1=CC=C(C(F)(F)F)C=C1,FC(F)(F)c1ccc(Cl)cc1.Cc1ccc(N)cc1.O=S(=O)(O[Pd...,[CH3:12][c:13]1[cH:14][cH:15][c:16]([NH2:17])[...,"{(88, 89, 0.0), (23, 24, 0.0), (33, 34, 0.0), ..."
3953,CC(C)C(C=C(C(C)C)C=C1C(C)C)=C1C2=C(P([C@@]3(C[...,C1(N(CC2=CC=CC=C2)CC3=CC=CC=C3)=CC=NO1,CN(C)P(N(C)C)(N(C)C)=NP(N(C)C)(N(C)C)=NCC,ClC1=CC=C(C(F)(F)F)C=C1,FC(F)(F)c1ccc(Cl)cc1.Cc1ccc(N)cc1.O=S(=O)(O[Pd...,[CH3:12][c:13]1[cH:14][cH:15][c:16]([NH2:17])[...,"{(53, 54, 0.0), (88, 89, 0.0), (23, 24, 0.0), ..."


In [18]:
df_doyle.drop(['Ligand','Additive','Base','Aryl halide'],inplace=True, axis=1)

In [19]:
df_doyle

,rxn,mapped_rxn,edit
0,Clc1ccccn1.Cc1ccc(N)cc1.O=S(=O)(O[Pd]1c2ccccc2...,[CH3:105][c:106]1[cH:107][c:108]([CH3:109])[o:...,"{(53, 54, 0.0), (86, 87, 0.0), (88, 89, 0.0), ..."
1,Brc1ccccn1.Cc1ccc(N)cc1.O=S(=O)(O[Pd]1c2ccccc2...,[Br:1][c:2]1[cH:3][cH:4][cH:5][cH:6][n:7]1.[CH...,"{(53, 54, 0.0), (86, 87, 0.0), (88, 89, 0.0), ..."
2,CCc1ccc(I)cc1.Cc1ccc(N)cc1.O=S(=O)(O[Pd]1c2ccc...,[CH3:10][c:11]1[cH:12][cH:13][c:14]([NH2:15])[...,"{(53, 54, 0.0), (86, 87, 0.0), (23, 24, 0.0), ..."
3,FC(F)(F)c1ccc(Cl)cc1.Cc1ccc(N)cc1.O=S(=O)(O[Pd...,[CH3:12][c:13]1[cH:14][cH:15][c:16]([NH2:17])[...,"{(88, 89, 0.0), (23, 24, 0.0), (33, 34, 0.0), ..."
4,COc1ccc(Cl)cc1.Cc1ccc(N)cc1.O=S(=O)(O[Pd]1c2cc...,[CH3:10][c:11]1[cH:12][cH:13][c:14]([NH2:15])[...,"{(53, 54, 0.0), (86, 87, 0.0), (88, 89, 0.0), ..."
...,...,...,...
3950,COc1ccc(Br)cc1.Cc1ccc(N)cc1.O=S(=O)(O[Pd]1c2cc...,[CH3:10][c:11]1[cH:12][cH:13][c:14]([NH2:15])[...,"{(53, 54, 0.0), (86, 87, 0.0), (23, 24, 0.0), ..."
3951,COc1ccc(Br)cc1.Cc1ccc(N)cc1.O=S(=O)(O[Pd]1c2cc...,[CH3:10][c:11]1[cH:12][cH:13][c:14]([NH2:15])[...,"{(86, 87, 0.0), (50, 52, 0.0), (88, 89, 0.0), ..."
3952,FC(F)(F)c1ccc(Cl)cc1.Cc1ccc(N)cc1.O=S(=O)(O[Pd...,[CH3:12][c:13]1[cH:14][cH:15][c:16]([NH2:17])[...,"{(88, 89, 0.0), (23, 24, 0.0), (33, 34, 0.0), ..."
3953,FC(F)(F)c1ccc(Cl)cc1.Cc1ccc(N)cc1.O=S(=O)(O[Pd...,[CH3:12][c:13]1[cH:14][cH:15][c:16]([NH2:17])[...,"{(53, 54, 0.0), (88, 89, 0.0), (23, 24, 0.0), ..."


In [20]:
df_doyle['reactant'] = df_doyle['mapped_rxn'].apply(lambda react:react.split('>')[0])
df_doyle['product'] = df_doyle['mapped_rxn'].apply(lambda react:react.split('>')[2])

In [22]:

df = df_doyle[["reactant", "product", "rxn", "mapped_rxn", "edit"]]

In [23]:
df

,reactant,product,rxn,mapped_rxn,edit
0,[CH3:105][c:106]1[cH:107][c:108]([CH3:109])[o:...,[CH3:2][c:3]1[cH:4][cH:5][c:6]([NH:7][c:8]2[cH...,Clc1ccccn1.Cc1ccc(N)cc1.O=S(=O)(O[Pd]1c2ccccc2...,[CH3:105][c:106]1[cH:107][c:108]([CH3:109])[o:...,"{(53, 54, 0.0), (86, 87, 0.0), (88, 89, 0.0), ..."
1,[Br:1][c:2]1[cH:3][cH:4][cH:5][cH:6][n:7]1.[CH...,[CH3:2][c:3]1[cH:4][cH:5][c:6]([NH:7][c:8]2[cH...,Brc1ccccn1.Cc1ccc(N)cc1.O=S(=O)(O[Pd]1c2ccccc2...,[Br:1][c:2]1[cH:3][cH:4][cH:5][cH:6][n:7]1.[CH...,"{(53, 54, 0.0), (86, 87, 0.0), (88, 89, 0.0), ..."
2,[CH3:10][c:11]1[cH:12][cH:13][c:14]([NH2:15])[...,[CH3:1][CH2:2][c:3]1[cH:4][cH:5][c:6]([NH:15][...,CCc1ccc(I)cc1.Cc1ccc(N)cc1.O=S(=O)(O[Pd]1c2ccc...,[CH3:10][c:11]1[cH:12][cH:13][c:14]([NH2:15])[...,"{(53, 54, 0.0), (86, 87, 0.0), (23, 24, 0.0), ..."
3,[CH3:12][c:13]1[cH:14][cH:15][c:16]([NH2:17])[...,[F:1][C:14]([F:3])([F:4])[c:13]1[cH:12][cH:11]...,FC(F)(F)c1ccc(Cl)cc1.Cc1ccc(N)cc1.O=S(=O)(O[Pd...,[CH3:12][c:13]1[cH:14][cH:15][c:16]([NH2:17])[...,"{(88, 89, 0.0), (23, 24, 0.0), (33, 34, 0.0), ..."
4,[CH3:10][c:11]1[cH:12][cH:13][c:14]([NH2:15])[...,[CH3:1][O:2][c:3]1[cH:4][cH:5][c:6]([NH:15][c:...,COc1ccc(Cl)cc1.Cc1ccc(N)cc1.O=S(=O)(O[Pd]1c2cc...,[CH3:10][c:11]1[cH:12][cH:13][c:14]([NH2:15])[...,"{(53, 54, 0.0), (86, 87, 0.0), (88, 89, 0.0), ..."
...,...,...,...,...,...
3950,[CH3:10][c:11]1[cH:12][cH:13][c:14]([NH2:15])[...,[CH3:1][O:2][c:3]1[cH:4][cH:5][c:6]([NH:15][c:...,COc1ccc(Br)cc1.Cc1ccc(N)cc1.O=S(=O)(O[Pd]1c2cc...,[CH3:10][c:11]1[cH:12][cH:13][c:14]([NH2:15])[...,"{(53, 54, 0.0), (86, 87, 0.0), (23, 24, 0.0), ..."
3951,[CH3:10][c:11]1[cH:12][cH:13][c:14]([NH2:15])[...,[CH3:1][O:2][c:3]1[cH:4][cH:5][c:6]([NH:15][c:...,COc1ccc(Br)cc1.Cc1ccc(N)cc1.O=S(=O)(O[Pd]1c2cc...,[CH3:10][c:11]1[cH:12][cH:13][c:14]([NH2:15])[...,"{(86, 87, 0.0), (50, 52, 0.0), (88, 89, 0.0), ..."
3952,[CH3:12][c:13]1[cH:14][cH:15][c:16]([NH2:17])[...,[F:1][C:14]([F:3])([F:4])[c:13]1[cH:12][cH:11]...,FC(F)(F)c1ccc(Cl)cc1.Cc1ccc(N)cc1.O=S(=O)(O[Pd...,[CH3:12][c:13]1[cH:14][cH:15][c:16]([NH2:17])[...,"{(88, 89, 0.0), (23, 24, 0.0), (33, 34, 0.0), ..."
3953,[CH3:12][c:13]1[cH:14][cH:15][c:16]([NH2:17])[...,[F:1][C:14]([F:3])([F:4])[c:13]1[cH:12][cH:11]...,FC(F)(F)c1ccc(Cl)cc1.Cc1ccc(N)cc1.O=S(=O)(O[Pd...,[CH3:12][c:13]1[cH:14][cH:15][c:16]([NH2:17])[...,"{(53, 54, 0.0), (88, 89, 0.0), (23, 24, 0.0), ..."
